In [50]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)

In [68]:
boat_df = pd.read_csv('Database/boat.csv')
div_df = pd.read_csv('Database/dividend.csv')
motor_df = pd.read_csv('Database/moter.csv')
player_df = pd.read_csv('Database/player.csv')
rank_df = pd.read_csv('Database/rank.csv')
schedule_df = pd.read_csv('Database/schedule.csv', low_memory=False)

In [17]:
schedule_df.shape

(104130, 42)

In [18]:
df_1 = schedule_df.copy()
boat_df['BOAT_NO'] = boat_df['BOAT_NO'].apply(lambda x: int(x[-2:]))

for col in boat_df.columns:
    if col != 'BASE_YEAR' and col != 'BOAT_NO':
        df_1[f'{col}_BOAT'] = np.nan

for i, row in df_1.iterrows():
    condition = (boat_df['BASE_YEAR'] == row['RACE_YEAR']) & (boat_df['BOAT_NO'] == row['BOAT_NO'])
    if condition.any():
        matching_row = boat_df[condition].iloc[0]
        for col in matching_row.index:
            if col != 'BASE_YEAR' and col != 'BOAT_NO':
                df_1.at[i, f'{col}_BOAT'] = matching_row[col]

In [19]:
df_1.shape

(104130, 51)

In [21]:
df_2 = df_1.copy()
motor_df['MOTOR_NO'] = motor_df['MOTOR_NO'].apply(lambda x: int(x[-2:]))

for col in motor_df.columns:
    if col != 'RACE_YEAR' and col != 'MOTOR_NO':
        df_2[f'{col}_MOTOR'] = np.nan

for i, row in df_2.iterrows():
    condition = (motor_df['RACE_YEAR'] == row['RACE_YEAR']) & (motor_df['MOTOR_NO'] == row['MOTOR_NO'])
    if condition.any():
        matching_row = motor_df[condition].iloc[0]
        for col in matching_row.index:
            if col != 'RACE_YEAR' and col != 'MOTOR_NO':
                df_2.at[i, f'{col}_MOTOR'] = matching_row[col]

In [22]:
df_2.shape

(104130, 60)

In [52]:
df_3 = df_2.copy()
rank_df = rank_df.iloc[:, :7]
rank_df.iloc[:, -3:] = rank_df.iloc[:, -3:].applymap(lambda x: str(x)[1:])

merged_df = df_3.merge(rank_df, how='left',
                       left_on=['RACE_YEAR', 'TME_VALUE', 'DAY_ORD_VALUE', 'RACE_NO'],
                       right_on=['RACE_YEAR', 'TME_VALUE', 'DAY_ORD_VALUE', 'RACE_NO'])

df_3['RANK'] = np.where(merged_df['PLAYER_NM'] == merged_df['N1R_PLAYER_NM'], 1,
                        np.where(merged_df['PLAYER_NM'] == merged_df['N2R_PLAYER_NM'], 2,
                                 np.where(merged_df['PLAYER_NM'] == merged_df['N3R_PLAYER_NM'], 3, -1)))

In [54]:
df_3.shape

(104130, 61)

In [58]:
df_4 = df_3.copy()

for col in player_df.columns:
    if col != 'RACE_YEAR' and col != 'PLAYER_NM':
        df_4[f'{col}_PLAYER'] = np.nan

for i, row in df_4.iterrows():
    condition = (player_df['RACE_YEAR'] == row['RACE_YEAR']) & (player_df['PLAYER_NM'] == row['PLAYER_NM'])
    if condition.any():
        matching_row = player_df[condition].iloc[0]
        for col in matching_row.index:
            if col != 'RACE_YEAR' and col != 'PLAYER_NM':
                df_4.at[i, f'{col}_PLAYER'] = matching_row[col]

In [59]:
df_4.shape

(104130, 78)

In [69]:
df_5=df_4.copy()
mask = div_df['OPER_INSTT_NM'] == '경정'
div_df = div_df[mask]
div_df.rename(columns={'PRACE_NO': 'RACE_NO'}, inplace=True)

df_5 = df_5.merge(div_df, how='left',
                       left_on=['RACE_YEAR', 'TME_VALUE', 'DAY_ORD_VALUE', 'RACE_NO'],
                       right_on=['BASE_YEAR', 'TME_VALUE', 'DAY_ORD_VALUE', 'RACE_NO'])

In [70]:
df_5.shape

(104130, 89)

In [71]:
df_5.to_csv('Database/train.csv')

# 여기서부터 새로 전처리 시작

In [82]:
data=pd.read_csv('Database/train.csv', index_col=0, low_memory=False)

## 배치로 묶기 위해 필요.
'RACE_YEAR', 경주년도
 'TME_VALUE', 회차값
 'DAY_ORD_VALUE', 일차값
 'RACE_NO', 경주번호

## 타겟칼럼 1,2,3까지는 순위가 있고 그 밑은 -1로 처리
 'RANK', 순위

## 필요없어 보이는 column.
 'RACE_GRAD_NM', 일반경주 or 특별경주
 'DPTR_TIME_VALUE', 출발시간 값/flying start 값 있어서 필요없어보임.
 'JUNGBUN_NM', 정번명/선수고유번호라는데 필요없어보임.
 'COLOR_NM', 색상명
 'PLAYER_NM', 선수명
 'THDT_PRACE_RACE_NO', 금일출주경주번호
 'BEF1_BRDNG_PLAYER_CKSN_VALUE', 전탑승선수
 'MOTOR_NO', 모터번호/ moter_df 다 합쳐서 필요없을듯
 'BOAT_NO', 보트번호/ boat_df 다 합쳐서 필요없을듯

## 중요한 column.
 'PLAYER_GRAD_NM', 선수등급 중요
 'SEXDSTN_FLAG_NM', 성별 아마도 중요
 'AGE_VALUE', 나이 별로 안중요
 'BDWGH_MESURE_VALUE', 몸무게 중요
 
 'LATST_SIX_TME_AVRG_CKSN_SCORE_VALUE', 최근6회차평균착순점수값
 'LATST_SIX_TME_AVRG_GTSR_VALUE', 최근6회차평균득점값
 'LATST_SIX_TME_VICTRY_RT_VALUE', 최근6회차승리율값
 'LATST_SIX_TME_YUNDAE_RT_TWO_VALUE', 최근6회차연대율2값
 'LATST_SIX_TME_YUNDAE_RT_THREE_VALUE', 최근6회차연대율3값
 'LATST_SIX_TME_AVGST_VALUE', 최근6회차평균ST값
 'LATST_EIGHT_RACE_CKSN_VALUE', 최근8경주착순값
 
 'FLYING_LATE_CO_VALUE', 플라잉 or 레이트 스타트 여부
 'AVRG_ACDTSCRE_VALUE', 평균사고점
 
 'FYER_ACCMLT_SCRE_AVRG_CKSN_SCORE_VALUE', 년간누적성적평균착순점수값
 'FYER_ACCMLT_SCRE_YUNDAE_RT_VALUE', 년간누적성적연대율값
 
 
## 좉같이 되어있어서 해석을 못함. 어떻게 쓸지 모르겠는 column
 'BEF1_DAY_SCRE_VALUE', 전일성적값
 'SIX_MONTH_PRACE_NCNT_VALUE', 6개월출주횟수값
 'SIX_MONTH_YUNDAE_RT_COSE1_PRACE_CO_VALUE', 6개월연대율1코스출주수값
 'SIX_MONTH_YUNDAE_RT_COSE2_PRACE_CO_VALUE', 6개월연대율2코스출주수값
 'SIX_MONTH_YUNDAE_RT_COSE3_PRACE_CO_VALUE', 6개월연대율3코스출주수값
 'SIX_MONTH_YUNDAE_RT_COSE4_PRACE_CO_VALUE', 6개월연대율4코스출주수값
 'SIX_MONTH_YUNDAE_RT_COSE5_PRACE_CO_VALUE', 6개월연대율5코스출주수값
 'SIX_MONTH_YUNDAE_RT_COSE6_PRACE_CO_VALUE', 6개월연대율6코스출주수값
 

## 출주표에 원래 있던 모터 and 보트 정보
 'MOTOR_AVRG_CKSN_SCORE_VALUE', 모터평균착순점수값
 'MOTOR_TWO_YUNDAE_RT_VALUE', 모터2연대율값
 'MOTOR_THREE_YUNDAE_RT_VALUE', 모터3연대율값

 'BOAT_AVRG_CKSN_SCORE_VALUE', 보트평균착순점수값
 'BOAT_YUNDAE_RT_VALUE', 보트연대율값
 'DPTR_MTH_CD', 출발방법코드
 
----------------------------------------------------------------------------------------
이 밑 부턴 실제 test셋을 구할수 있는지 모르겠음

## 출주표에 없는 보트정보
  데이터 스키마가 없어서 못적음.
 'PRACE_NCNT_BOAT',
 'N1CK_NCNT_BOAT',
 'N2CK_NCNT_BOAT',
 'N3CK_NCNT_BOAT',
 'AVRG_CKSN_SCORE_VALUE_BOAT',
 'YUNDAE_RT_BOAT',
 'BEST_SAIL_TIME_VALUE_BOAT',
 'N2JU_BEST_TIME_VALUE_BOAT',
 'N3JU_BEST_TIME_VALUE_BOAT',
 
## 출주표에 없는 모터정보
 'PRACE_NCNT_MOTOR', 출주횟수
 'N1CK_NCNT_MOTOR', 1착횟수
 'N2CK_NCNT_MOTOR', 2착횟수
 'N3CK_NCNT_MOTOR', 3착횟수
 'AVRG_CKSN_SCORE_VALUE_MOTOR', 평균착순점수값
 'YUNDAE_RT_MOTOR', 연대율
 'BEST_SAIL_TIME_VALUE_MOTOR', 최고항주시간값
 'N2JU_BEST_TIME_VALUE_MOTOR', 2주회최고시간값
 'N3JU_BEST_TIME_VALUE_MOTOR', 3주회최고시간값
 
## 출주표에 없는 선수정보
  데이터 스키마가 없어서 못적음.
 'PLAYER_REGIST_NO_PLAYER',
 'PRACE_NCNT_PLAYER',
 'PRACE_1R_NCNT_PLAYER',
 'PRACE_2R_NCNT_PLAYER',
 'PRACE_3R_NCNT_PLAYER',
 'PRACE_4R_NCNT_PLAYER',
 'PRACE_5R_NCNT_PLAYER',
 'PRACE_6R_NCNT_PLAYER',
 'AVRG_CKSN_SCORE_VALUE_PLAYER',
 'AVRG_ACDNT_SCORE_VALUE_PLAYER',
 'AVRG_SCORE_VALUE_PLAYER',
 'AVRG_BEGIN_TIME_VALUE_PLAYER',
 'VICTRY_RT_PLAYER',
 'YUNDAE_RT_PLAYER',
 'THREE_YUNDAE_RT_PLAYER',
 'PLAYER_GRAD_NM_PLAYER',
 'PLAYER_PN_NO_PLAYER',
 
## 배당정보인데 순위예측할거면 필요없지만 일단 붙여놓음.
 'OPER_INSTT_NM', 운영기관명
 'BASE_YEAR', 기준년도
 'RACE_DE', 경주일자
 'DANSEONG_VALUE', 단승값
 'YNGSEONG_ONE_VALUE', 연승1값
 'YNGSEONG_TWO_VALUE', 연승2값
 'SSANGSEONG_VALUE', 쌍승값
 'BOKSEONG_VALUE', 복승값
 'SAMBOKSEONG_VALUE', 삼복승값
 'SAMSSANGSEONG_DVDN_RT', 삼쌍승배당율
 'SSANBOKSEONG_DVDN_RT' 쌍복승배당율